In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import numpy as np
max_long = 25

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [5]:
!pip install unidecode

In [6]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [8]:
import token_glove2
token_glove2.create_list, token_glove2.token_ize, token_glove2.voc_token, token_glove2.dict_token, token_glove2.token_tweet, token_glove2.glove_vector

(<function token_glove2.create_list>,
 <function token_glove2.token_ize>,
 <function token_glove2.voc_token>,
 <function token_glove2.dict_token>,
 <function token_glove2.token_tweet>,
 <function token_glove2.glove_vector>)

In [9]:
import metrics
metrics.recall_m, metrics.precision_m, metrics.f1_m

(<function metrics.recall_m>,
 <function metrics.precision_m>,
 <function metrics.f1_m>)

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [11]:
df = data.get_data()
cleaned_df = data.clean_data(df)

In [12]:
text = token_glove2.create_list(cleaned_df['text'])
token = token_glove2.token_ize(text)
X,token_izer = token_glove2.token_tweet(text, token)

In [14]:
word_vector_matrix = token_glove2.glove_vector(token)

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
vocab_size = token_glove2.voc_token(token)
vec_size = 200

In [18]:
y = cleaned_df['target']

In [19]:
model = Sequential()

es = EarlyStopping(patience=5,restore_best_weights= True)

model.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model.add(Conv1D(128, 16, activation ='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate = 0.0001),metrics=['accuracy'])
history = model.fit(X, y, batch_size=16, epochs = 22, validation_split=0.1, callbacks = [es])

Epoch 1/22
429/429 [==============================] - 17s 37ms/step - loss: 0.6849 - accuracy: 0.5663 - val_loss: 0.6293 - val_accuracy: 0.7480
Epoch 2/22
429/429 [==============================] - 16s 36ms/step - loss: 0.5950 - accuracy: 0.7043 - val_loss: 0.5211 - val_accuracy: 0.8045
Epoch 3/22
429/429 [==============================] - 15s 36ms/step - loss: 0.5306 - accuracy: 0.7538 - val_loss: 0.4881 - val_accuracy: 0.8071
Epoch 4/22
429/429 [==============================] - 15s 36ms/step - loss: 0.4969 - accuracy: 0.7759 - val_loss: 0.4732 - val_accuracy: 0.8071
Epoch 5/22
429/429 [==============================] - 16s 37ms/step - loss: 0.4650 - accuracy: 0.7942 - val_loss: 0.4556 - val_accuracy: 0.8058
Epoch 6/22
429/429 [==============================] - 16s 38ms/step - loss: 0.4447 - accuracy: 0.8101 - val_loss: 0.4522 - val_accuracy: 0.8097
Epoch 7/22
429/429 [==============================] - 16s 38ms/step - loss: 0.4182 - accuracy: 0.8240 - val_loss: 0.4458 - val_accuracy:

#Phase de Test

In [21]:
df = data.get_data(test=True)
test_df = data.clean_data(df)

In [22]:
text_test = token_glove2.create_list(test_df['text'])
encoded_text_test = token_izer.texts_to_sequences(text_test)
X_test = pad_sequences(encoded_text_test, maxlen=max_long, padding='post')

In [23]:
results = model.predict(X_test)

In [24]:
results

array([[0.6388107 ],
       [0.84428406],
       [0.87397337],
       ...,
       [0.927103  ],
       [0.7617718 ],
       [0.5499463 ]], dtype=float32)

In [25]:
def arrondi(n):
  if n >= 0.5:
    return 1
  else :
    return 0

In [26]:
to_submit = [arrondi(x[0]) for x in results]

In [27]:
to_submit_df = pd.DataFrame(to_submit)

In [28]:
to_submit_df.rename(columns={0: 'target'}, inplace=True)

In [29]:
to_submit_df['id'] = test_df.id

In [30]:
to_submit_df = to_submit_df.set_index('id')

In [31]:
to_submit_df['target'].value_counts()

0    2053
1    1210
Name: target, dtype: int64

In [32]:
to_submit_df.to_csv('/content/gdrive/My Drive/submission8.csv')

In [33]:
model.save('/content/gdrive/My Drive/CNN_camille_version_finale')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/CNN_camille_version_finale/assets
